# Demyst Interactive Quickstart

**The Scientific Integrity Linter for ML and Data Science**

---

## The Problem: What if 999 out of 1000 AI agents are safe... but one is rogue?

```python
swarm_alignment = [1.0, 1.0, 1.0, ..., 1.0, 0.0]  # 999 perfect, 1 rogue
mean_alignment = np.mean(swarm_alignment)         # = 0.999
if mean_alignment > 0.99:
    deploy_swarm()  # CATASTROPHIC FAILURE
```

The mean hides the rogue agent. This is a **computational mirage** - and it's just one of many scientific integrity issues Demyst catches.

---

**This notebook covers:**
1. Installation & Setup
2. Computational Mirages (Swarm Collapse)
3. Data Leakage Detection
4. P-Hacking Prevention
5. Unit/Dimensional Analysis
6. Deep Learning Integrity
7. Auto-Fix Capabilities
8. Full Pipeline Demo

## 1. Installation & Setup

In [ ]:
# Install Demyst (run once)
# !pip install demyst

# For development/local install:
# !pip install -e ..

# Verify installation
!demyst --version

In [ ]:
# Import Demyst components
import numpy as np
import ast
import sys
import os

# Add parent directory to path for local development
sys.path.insert(0, os.path.abspath('..'))

from demyst import MirageDetector, LeakageHunter, HypothesisGuard, UnitGuard, TensorGuard

print("Available guards:")
print("  - MirageDetector: Detects variance-destroying operations")
print("  - LeakageHunter: Detects train/test data leakage")
print("  - HypothesisGuard: Detects p-hacking and statistical issues")
print("  - UnitGuard: Checks dimensional consistency")
print("  - TensorGuard: Checks deep learning integrity")

---

## 2. Computational Mirages: The Swarm Collapse Problem

A **computational mirage** occurs when an operation destroys statistical information, leading to incorrect conclusions.

In [ ]:
# THE DANGEROUS CODE
# This is the exact pattern Demyst catches

def analyze_swarm_safety(agent_count=1000):
    """Analyze AI agent swarm - but with a hidden danger."""
    
    # 999 agents are perfectly aligned (1.0)
    # 1 agent is ROGUE (0.0) - injected with a jailbreak
    swarm_alignment = np.ones(agent_count)
    swarm_alignment[-1] = 0.0
    
    # THE MIRAGE: Mean hides the rogue agent
    mean_alignment = np.mean(swarm_alignment)  # Demyst flags this!
    
    print(f"Swarm Size: {agent_count}")
    print(f"Average Alignment: {mean_alignment:.4f}")
    
    if mean_alignment > 0.99:
        print("\nDECISION: Deploy swarm (looks safe!)")
        print("REALITY: Rogue agent causes cascade failure")
    
    return mean_alignment

# Run the dangerous code
analyze_swarm_safety()

In [ ]:
# DEMYST DETECTION (CLI)
# Let's see what Demyst finds

!demyst mirage ../examples/swarm_collapse.py

In [ ]:
# DEMYST DETECTION (Python API)

dangerous_code = '''
import numpy as np
scores = np.ones(1000)
scores[-1] = 0.0
mean_score = np.mean(scores)  # MIRAGE: hides the outlier
'''

tree = ast.parse(dangerous_code)
detector = MirageDetector()
detector.visit(tree)

print(f"Mirages detected: {len(detector.mirages)}")
for m in detector.mirages:
    print(f"  Line {m['line']}: {m['type']} - {m['function']}()")

---

## 3. Data Leakage: The #1 ML Error

**"If test data touches training, your benchmark is a lie."**

Data leakage is the most common error in machine learning, causing:
- Inflated accuracy metrics
- Results that don't replicate
- Models that fail in production

In [ ]:
# THE WRONG WAY (Leakage!)

leaky_code = '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
X, y = load_data()

# WRONG: Fit scaler on ALL data (including test!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Leakage here!

# Split AFTER preprocessing - damage is done
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)
'''

print("THE WRONG WAY:")
print(leaky_code)

In [ ]:
# DEMYST LEAKAGE DETECTION

hunter = LeakageHunter()
result = hunter.analyze(leaky_code)

print(f"Violations found: {result['summary']['total_violations']}")
print(f"Verdict: {result['summary']['verdict']}")
print()

for v in result['violations']:
    print(f"Line {v['line']}: {v['type']}")
    print(f"  {v['description'][:100]}...")

In [ ]:
# THE CORRECT WAY

correct_code = '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
X, y = load_data()

# CORRECT: Split FIRST
X_train, X_test, y_train, y_test = train_test_split(X, y)

# CORRECT: Fit on train ONLY, transform both
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # fit on train
X_test = scaler.transform(X_test)        # transform only!
'''

result = hunter.analyze(correct_code)
print(f"Verdict: {result['summary']['verdict']}")

In [ ]:
# CLI: Run on the full example
!demyst leakage ../examples/ml_data_leakage.py

---

## 4. P-Hacking: When Statistics Lie

Running 100 statistical tests at p < 0.05 means ~5 will be "significant" by chance alone.
This is called **p-hacking** and it's a major source of irreproducible research.

In [ ]:
# THE DANGEROUS PATTERN

phacking_code = '''
from scipy import stats
import numpy as np

# Test 1000 genes for differential expression
results = []
for gene in genes:
    t_stat, p_value = stats.ttest_ind(control[gene], treatment[gene])
    if p_value < 0.05:  # Uncorrected threshold!
        results.append(gene)  # Conditional reporting

print(f"Significant genes: {len(results)}")  # ~50 expected by chance!
'''

print("THE P-HACKING PATTERN:")
print(phacking_code)

In [ ]:
# DEMYST DETECTION

guard = HypothesisGuard()
result = guard.analyze_code(phacking_code)

print(f"Issues found: {len(result.get('issues', []))}")
for issue in result.get('issues', []):
    print(f"  - {issue.get('type', 'unknown')}: {issue.get('description', '')[:80]}...")

In [ ]:
# CLI: Run on biology example
!demyst hypothesis ../examples/biology_gene_expression.py

---

## 5. Unit/Dimensional Analysis

Adding meters to seconds is physically nonsensical. Demyst catches these errors.

In [ ]:
# THE PHYSICS ERROR

physics_code = '''
# Particle trajectory calculation
position = initial_position + velocity  # ERROR: position + velocity?
distance = time + offset  # ERROR: distance = time + ???
force = energy  # ERROR: force [MLT^-2] != energy [ML^2T^-2]
'''

guard = UnitGuard()
result = guard.analyze(physics_code)

print(f"Unit violations: {len(result.get('violations', []))}")
for v in result.get('violations', []):
    print(f"  Line {v.get('line', '?')}: {v.get('description', '')}")

In [ ]:
# CLI: Run on physics example
!demyst units ../examples/physics_kinematics.py

---

## 6. Deep Learning Integrity: Gradient Death

**"If the gradient dies in silence, the model learns nothing."**

Deep sigmoid/tanh networks suffer from vanishing gradients - early layers stop learning.

In [ ]:
# THE GRADIENT DEATH PATTERN

dl_code = '''
import torch.nn as nn

class DeepSigmoidNetwork(nn.Module):
    """10 layers of Sigmoid = gradient death"""
    
    def __init__(self):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Linear(100, 50) for _ in range(10)
        ])
        self.activation = nn.Sigmoid()  # Saturating activation!
    
    def forward(self, x):
        for layer in self.layers:
            x = self.activation(layer(x))  # Gradient death chain
        return x
'''

guard = TensorGuard()
result = guard.analyze(dl_code)

print(f"Gradient issues: {len(result.get('gradient_issues', []))}")
for issue in result.get('gradient_issues', []):
    print(f"  {issue['type']}: {issue['description'][:80]}...")

In [ ]:
# CLI: Run on deep learning example
!demyst tensor ../examples/deep_learning_gradient_death.py

---

## 7. Full Pipeline: Analyze Your Code

Run all checks at once:

In [ ]:
# Run full analysis on all examples
!demyst analyze ../examples/ --format markdown

In [ ]:
# Generate an integrity report
!demyst report ../examples/ --format json

---

## 8. Try It Yourself!

Paste your code below and see what Demyst finds:

In [ ]:
# YOUR CODE HERE
# Paste your Python code and run this cell to analyze it

your_code = '''
# Paste your code here
import numpy as np

data = np.random.randn(100)
result = np.mean(data)
'''

# Write to temp file and analyze
with open('/tmp/your_code.py', 'w') as f:
    f.write(your_code)

!demyst analyze /tmp/your_code.py

---

## Next Steps

1. **Add to pre-commit:** See `examples/configs/.pre-commit-config.minimal.yaml`
2. **Add to CI:** See `examples/configs/demyst-ci.yml`
3. **Configure for your domain:** See `examples/configs/.demystrc.yaml`
4. **Full documentation:** See `docs/usage.md`

---

*"The first principle is that you must not fool yourself—and you are the easiest person to fool."* - Richard Feynman